In [0]:
%pylab inline

In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

In [0]:
import geopandas as gpd
import requests
import json
from pandas.io.json import json_normalize
from itertools import islice
from shapely.geometry import Polygon, LineString
import matplotlib.pyplot as plt

In [0]:
# This function returns the fields (columns) of a given data layer number
def layer_fields(layer_number):
    layer = 'https://gisservices.noord-holland.nl/ags/rest/services/extern/pnh_areaal/MapServer/' + str(layer_number)
    query = layer +'?f=pjson'
    r = requests.get(query)
    info = json_normalize(json.loads(r.content))
    info = json_normalize(info.fields[0])
    return info

In [0]:
# This function returns all objectIDs of a given data layer number within an envelope
##### It seems that the envelope is ignored, every envelope gives the same amount of objectIDs. Find out why!
def layer_objectIDs(layer_number, envelope):
    layer = 'https://gisservices.noord-holland.nl/ags/rest/services/extern/pnh_areaal/MapServer/' + str(layer_number)
    query = layer +'/query?where=&text=&objectIds=&time=&geometry='+ envelope + '&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=&returnGeometry=false&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&having=&returnIdsOnly=true&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentOnly=false&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&f=pjson'
    r =  requests.get(query, verify=False)
    df = json_normalize(json.loads(r.content))
    objectIDs= df.objectIds[0]
    return objectIDs

In [0]:
# This function returns the data of the given fields of a given layer number, for the given objectIDs
def layer_data(layer_number, fields, objectIDs):
    fieldsstring = ''
    for f in range(0,len(fields)):
        field = fields[f]
        if fieldsstring == '':
            fieldsstring = field
        else:
            fieldsstring = fieldsstring + '%2C+' + field
    layer = 'https://gisservices.noord-holland.nl/ags/rest/services/extern/pnh_areaal/MapServer/' + str(layer_number)
    #query =layer+'/query?where=&text=&objectIds='+ objectIDs +'&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields='+ fieldsstring + '&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&having=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentOnly=false&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&f=pjson'
    query =layer+'/query?where=&text=&objectIds='+ objectIDs +'&time=&geometry=&geometryType=&inSR=&spatialRel=&relationParam=&outFields='+ fieldsstring + '&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&having=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentOnly=false&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&f=pjson'

    r =  requests.get(query, verify=False)
    df = json_normalize(json.loads(r.content))
    df = json_normalize(df.features[0])
    return df

In [0]:
# This function divides the objectIDs in chunks
def chunk(objectIDs, size):
    objectIDs = iter(objectIDs)
    objectIDs_chunks = list(iter(lambda: tuple(islice(objectIDs, size)), ()))
    return objectIDs_chunks

In [0]:
layer_number = 111 # wegvak_v
#number = 5 # Gemeentegrenzen
#number = 112 # Wegdeel_v
#layer_number = 52 # ecopassage
all_fields = layer_fields(layer_number)
fields = all_fields['name']

In [0]:
fields

In [0]:
xmin = 94297.0340000019
xmax = 148229.3149
ymin = 465598.8116
ymax = 563639.916299999
envelope = str(xmin) + '%2C+' + str(ymin) + '%2C+' + str(xmax) + '%2C+' + str(ymax)
objectIDs = layer_objectIDs(layer_number, envelope)

In [0]:
envelope

In [0]:
len(objectIDs)

In [0]:
n = 100 # grootte van chunks
objectIDs_chunks = list(chunk(objectIDs, n))

In [0]:
number_chunks = len(objectIDs_chunks)
number_chunks

In [0]:
# Areaaldata downloaden per chunk
df = pd.DataFrame()
for i in range(0,number_chunks):
    chunk = objectIDs_chunks[i]
    objectIDs_chunk = ''
    for j in range(0,len(chunk)):
        objectID = str(chunk[j])
        if objectIDs_chunk == '':
            objectIDs_chunk = objectID
        else:
            objectIDs_chunk= objectIDs_chunk + '%2C+' + objectID
    data = layer_data(layer_number, fields, objectIDs_chunk)
    df = df.append(data)

In [0]:
df

In [0]:
df['attributes.TRAJECT'].unique()

In [0]:
# Stukje code van Sam om geometry te kunnen lezen.
# dit moet anders en mooier kunnen maar het werkt
def converter(row):
    # row index 0 want list is nested
    geopoints = [tuple(coords) for coords in row[0]]
    return geopoints

In [0]:
df['geometry.rings'] = df['geometry.rings'].astype(str)
#df['geometry.paths'] = df['geometry.paths'].astype(str)

In [0]:
# data is in string format -> eerst omzetten naar nested list dmv json.loads
df['geometry'] = df['geometry.rings'].apply(lambda x: Polygon(converter(json.loads(x))))
#df['geometry'] = df['geometry.paths'].apply(lambda x: Polygon(converter(json.loads(x))))

In [0]:
gdf = gpd.GeoDataFrame(df, crs='epsg:28992', geometry = 'geometry')

In [0]:
fig, ax = plt.subplots(1, 1, figsize = (10,10))
gdf.plot(ax=ax)

In [0]:
# Write recipe outputs
wegvakken_v = dataiku.Dataset("wegvakken_v")
wegvakken_v.write_with_schema(df)

In [0]:
# Recipe outputs
wegvakken_v = dataiku.Dataset("wegvakken_v")
wegvakken_v.write_with_schema(pandas_dataframe)